In [25]:

# Acknowledgment:
# This code was developed with assistance from ChatGPT, which provided initial guidance
# and code snippets. Significant modifications and improvements were made by me to 
# ensure the code meets the specific requirements of the project. All optimizations 
# and final adjustments were implemented independently.




# Usual Libraries USED FROM THE OTHER NOTEBOOK WE ADD MORE OR DELETE WHEN NECESSESARRY 
import pandas as pd
import numpy as np
import distutils
import ast

import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

# Librosa (the mother of audio files)
import librosa
import librosa.display
import IPython.display as ipd
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tqdm import tqdm

import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [ ]:


# Define the context features
context_features = {
    'video_id': tf.io.FixedLenFeature([], tf.string),
    'start_time_seconds': tf.io.FixedLenFeature([], tf.float32),
    'end_time_seconds': tf.io.FixedLenFeature([], tf.float32),
    'labels': tf.io.VarLenFeature(tf.int64)
}

# Define the sequence features
sequence_features = {
    'audio_embedding': tf.io.VarLenFeature(tf.string)
}

def _parse_function(example_proto):
    # Parse the example into context and sequence features
    context_data, sequence_data = tf.io.parse_single_sequence_example(
        example_proto,
        context_features=context_features,
        sequence_features=sequence_features
    )
    
    # Decode the audio embeddings from the byte strings
    audio_embeddings = tf.io.decode_raw(sequence_data['audio_embedding'].values, tf.uint8)
    audio_embeddings = tf.reshape(audio_embeddings, [-1, 128])  # Reshape to [time_steps, embedding_size]
    
    # Normalize the embeddings to the range [0, 1]
    audio_embeddings = tf.cast(audio_embeddings, tf.float32) / 255.0
    
    # Flatten the embedding to a 1D array
    flattened_embedding = tf.reshape(audio_embeddings, [-1])
    
    # Convert sparse labels to dense
    labels = tf.sparse.to_dense(context_data['labels'])
    
    return flattened_embedding, labels

def parse_tfrecords_to_dataframe(tfrecord_files):
    # Initialize an empty list to store embeddings and labels
    embeddings_list = []
    labels_list = []

    # Iterate through each TFRecord file in the folder
    for tfrecord_file in tqdm(tfrecord_files, desc="Processing TFRecord files"):
        raw_dataset = tf.data.TFRecordDataset(tfrecord_file)
        parsed_dataset = raw_dataset.map(_parse_function)

        # Iterate over the parsed dataset and collect embeddings and labels
        for audio_embedding, labels in parsed_dataset:
            # Convert to numpy arrays
            audio_embedding_np = audio_embedding.numpy()
            labels_np = labels.numpy()

            # Flatten the label array if necessary
            labels_np = labels_np.flatten() if len(labels_np.shape) > 0 else labels_np

            # Store the embedding and label
            embeddings_list.append(audio_embedding_np)
            labels_list.append(labels_np)

    # Create a DataFrame from the collected data
    df = pd.DataFrame({
        'embedding': embeddings_list,
        'labels': labels_list
    })

    return df

# Specify the folder containing TFRecord files
bal_tfrecord_folder = '.github\Data\\bal_train'
bal_tfrecord_files = [os.path.join(bal_tfrecord_folder, f) for f in os.listdir(bal_tfrecord_folder) if f.endswith('.tfrecord')]

eval_tfrecord_folder = '.github\Data\\eval'
eval_tfrecord_files = [os.path.join(eval_tfrecord_folder, f) for f in os.listdir(eval_tfrecord_folder) if f.endswith('.tfrecord')]
eval_df = parse_tfrecords_to_dataframe(eval_tfrecord_files)
# Parse the TFRecords into a DataFrame
bal_df = parse_tfrecords_to_dataframe(bal_tfrecord_files)




Processing TFRecord files: 100%|██████████| 1444/1444 [01:30<00:00, 16.01it/s]


In [ ]:
# Ensure the 'data' directory exists
output_folder = r'.github\Data'
os.makedirs(output_folder, exist_ok=True)

# Paths for saving the CSV files
eval_csv_path = os.path.join(output_folder, 'eval_df.csv')
bal_csv_path = os.path.join(output_folder, 'bal_df.csv')

# Save the DataFrames to CSV
eval_df.to_csv(eval_csv_path, index=False)
bal_df.to_csv(bal_csv_path, index=False)




Reshape them  embeddings

In [ ]:
# Parameters (adjust as necessary)
embedding_length = 128  # This should match the original dimension of each heatmap slice
sequence_length = 100  # Adjust to match the desired number of time steps per sequence

# Convert the embedding column to a numpy array and reshape each embedding
def reshape_embeddings(df, embedding_length, sequence_length):
    reshaped_embeddings = []
    for embedding in df['embedding']:
        # Ensure embedding is a numpy array
        embedding = np.array(embedding)
        
        # Reshape the embedding into (sequence_length, embedding_length)
        # If necessary, truncate or pad to the sequence_length
        if len(embedding) < sequence_length * embedding_length:
            # Pad with zeros if too short
            padding_length = sequence_length * embedding_length - len(embedding)
            embedding = np.pad(embedding, (0, padding_length), mode='constant')
        elif len(embedding) > sequence_length * embedding_length:
            # Truncate if too long
            embedding = embedding[:sequence_length * embedding_length]
        
        # Reshape to (sequence_length, embedding_length)
        embedding = embedding.reshape((sequence_length, embedding_length))
        reshaped_embeddings.append(embedding)

    # Convert to numpy array for model input
    reshaped_embeddings = np.array(reshaped_embeddings)
    return reshaped_embeddings

# Apply reshaping to embeddings
reshaped_bal_embeddings = reshape_embeddings(bal_df, embedding_length, sequence_length)
#reshaped_eval_embeddings = reshape_embeddings(eval_df, embedding_length, sequence_length)
# Output the reshaped array's shape to verify
print("Reshaped Embeddings Shape:", reshaped_bal_embeddings.shape)
#print("Reshaped Embeddings Shape:", reshaped_eval_embeddings.shape)

Reshaped Embeddings Shape: (7971, 100, 128)


In [20]:
# 1 means the notes is apart of the scale, and 0 means that it is not
# The twelve major scales are C C# D D# E F F# G G# A A# B
# EX. C major scale is C D E F G A B which is the first scale in the array
MAJOR_KEY_PROFILES = np.array([
    [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1],  # C Major
    [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],  # C# Major
    [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],  # D Major
    [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],  # Eb Major
    [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1],  # E Major
    [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],  # F Major
    [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],  # F# Major
    [1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0],  # G Major
    [0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],  # Ab Major
    [1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],  # A Major
    [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1],  # Bb Major
    [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0],  # B Major
])

def determine_key(chroma_aggregate):
    # Normalize the chroma vector
    chroma_aggregate = chroma_aggregate / np.linalg.norm(chroma_aggregate)

    # Compute similarity with each major key profile
    similarities = [np.dot(chroma_aggregate, profile) for profile in MAJOR_KEY_PROFILES]
    
    # Find the key with the highest similarity
    key_index = np.argmax(similarities)
    keys = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    return keys[key_index]

Splitting the data and training 

In [ ]:
import numpy as np
from scipy.stats import norm
#https://cs229.stanford.edu/proj2016/report/Mahieu-DetectingMusicalKeyWithSupervisedLearning-report.pdf
#https://www.kaggle.com/discussions/general/278766



def probabilistic_key_detection(chroma_profile, key_profiles):
    """
    Detects the most likely key using a probabilistic approach.
    
    Args:
        chroma_profile (np.array): A 12-dimensional chroma vector (normalized).
        key_profiles (np.array): A 12x12 matrix of major key templates.

    Returns:
        str: The predicted key name.
    """
    # make the sum of all the elements = 1 for consistency 
    chroma_profile = chroma_profile / np.sum(chroma_profile)

    likelihoods = []
    for key_profile in key_profiles:
        # Compute the likelihood by using a guassian noramlization method
        # It find the difference in squares, and then uses a guassian equation to find the likelihood of the key. 
        likelihood = np.exp(-0.5 * np.sum((chroma_profile - key_profile) ** 2))
        likelihoods.append(likelihood)
    
    # The sum of all probabilities will = 1, so that we can later find the highest probabability key
    likelihoods = np.array(likelihoods)
    likelihoods /= np.sum(likelihoods)

    # Identify the most probable key
    keys = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    # Assigns the highest value likelyhoods of the keys 
    most_likely_key = keys[np.argmax(likelihoods)]
    
    return most_likely_key, likelihoods


In [16]:
import librosa
import librosa.display

def extract_chroma_features(audio_embedding, sr=22050):
    """
    Extracts chroma features from an audio embedding.
    
    Args:
        audio_embedding (np.array): Flattened audio embedding representing a segment.
        sr (int): Sampling rate (default is 22050).
    
    Returns:
        np.array: Normalized 12-dimensional chroma feature.
    """
    # Ensure embedding is in range [-1, 1] as librosa expects audio signals
    audio_embedding = audio_embedding / np.max(np.abs(audio_embedding))

    # Extract chroma features using librosa
    chroma = librosa.feature.chroma_cqt(y=audio_embedding, sr=sr, n_chroma=12)

    # Average across time steps to get a single 12-dimensional vector
    chroma_avg = np.mean(chroma, axis=1)

    # Normalize the chroma profile to sum to 1
    chroma_normalized = chroma_avg / np.sum(chroma_avg)

    return chroma_normalized



In [28]:
# Example: Iterate through your dataframe and process each embedding
key_predictions = []
likelihood_distributions = []

for index, row in bal_df.iterrows():
    # Extract audio embedding
    audio_embedding = np.array(row['embedding'])  # Ensure the embedding is a numpy array

    # Extract chroma features from the embedding
    chroma_profile = extract_chroma_features(audio_embedding)

    # Detect the most likely key and its likelihood distribution
    predicted_key, likelihoods = probabilistic_key_detection(chroma_profile, MAJOR_KEY_PROFILES)

    # Store results
    key_predictions.append(predicted_key)
    likelihood_distributions.append(likelihoods)

# Add the predicted keys and likelihoods to the dataframe
bal_df['predicted_key'] = key_predictions
bal_df['likelihoods'] = likelihood_distributions

# Example output
print(bal_df[['embedding', 'predicted_key', 'likelihoods']].head())


                                           embedding predicted_key  \
0  [0.37254903, 0.47843137, 0.5764706, 0.05490196...             A   
1  [0.3137255, 0.70980394, 0.84705883, 0.6431373,...             A   
2  [0.41960785, 0.5882353, 0.34901962, 0.5411765,...             A   
3  [0.5529412, 0.3254902, 0.43137255, 0.69411767,...             F   
4  [0.4, 0.18431373, 0.6666667, 0.23137255, 0.533...             A   

                                         likelihoods  
0  [0.05636778845112223, 0.08669995534079632, 0.0...  
1  [0.056948113012707996, 0.08381732069689841, 0....  
2  [0.05644942824928668, 0.08634687890280632, 0.0...  
3  [0.05771204558336973, 0.08250187840304446, 0.0...  
4  [0.056701312957757065, 0.0845041332789606, 0.0...  


In [ ]:
print(bal_df.describe)
x_train = bal_df.drop['likelihoods', ]

<bound method NDFrame.describe of                                               embedding              labels  \
0     [0.37254903, 0.47843137, 0.5764706, 0.05490196...               [399]   
1     [0.3137255, 0.70980394, 0.84705883, 0.6431373,...            [0, 451]   
2     [0.41960785, 0.5882353, 0.34901962, 0.5411765,...           [27, 466]   
3     [0.5529412, 0.3254902, 0.43137255, 0.69411767,...        [0, 95, 137]   
4     [0.4, 0.18431373, 0.6666667, 0.23137255, 0.533...          [427, 431]   
...                                                 ...                 ...   
7966  [0.5647059, 0.6431373, 0.8039216, 0.4, 0.70980...             [0, 52]   
7967  [0.26666668, 0.17254902, 0.8352941, 0.3882353,...             [0, 50]   
7968  [0.34509805, 0.09803922, 0.654902, 0.39607844,...        [0, 26, 506]   
7969  [0.6784314, 0.105882354, 0.49411765, 0.3725490...               [247]   
7970  [0.7607843, 0.63529414, 0.9411765, 0.34117648,...  [27, 33, 137, 254]   

     key predicte